In [1]:
import pandas as pd

data= pd.read_csv("kc_house_data.csv")
data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [2]:
data=data.iloc[:,[2,5]].values

In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data = scaler.fit_transform(data)

In [4]:
x=data[:,0]
y=data[:,1]
print(y)

[-0.97983502  0.53363434 -1.42625404 ... -1.15404732 -0.52252773
 -1.15404732]


In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.10, random_state = 42)

In [6]:
batch_size = 32
n_minibatches = x_train.shape[0] // batch_size

def create_mini_batches(x_tr, y_tr, batch_size):
    mini_batches = [] 
    global n_minibatches
    for i in range(n_minibatches + 1): 
        X_mini = x_tr[i * batch_size : (i + 1) * batch_size]
        Y_mini = y_tr[i * batch_size : (i + 1) * batch_size]
        mini_batches.append((X_mini, Y_mini)) 
        if x_tr.shape[0] % batch_size != 0:
            X_mini = x_tr[i * batch_size: x_tr.shape[0]]
            Y_mini  = y_tr[i * batch_size: y_tr.shape[0]]
        mini_batches.append((X_mini,Y_mini)) 
    return mini_batches 

mini_batches_all = create_mini_batches(x_train, y_train, batch_size)

In [7]:
m=1
c = -1 
lr = 0.1 
delta_m = 1 
delta_c = 1 
max_iters = 10 
iters_count = 0 


def deriv(m_f, c_f, datax, datay):
    m_deriv = 0
    c_deriv = 0
    for i in range(datax.shape[0]):
        x, y = datax[i], datay[i]
        m_deriv += (y-m_f*x-c_f)*x
        c_deriv += (y-m_f*x-c_f)
        m_deriv = -m_deriv/len(datax)
        c_deriv = -c_deriv/len(datay)
    return m_deriv, c_deriv  


while iters_count < max_iters:
    for i in range(n_minibatches):
        x_b, y_b = mini_batches_all[i]
        delta_m, delta_c = deriv(m, c, x_b, y_b)
        delta_m = -lr * delta_m
        delta_c = -lr * delta_c
        m += delta_m
        c += delta_c
    iters_count += 1
    print(f"Iteration: {iters_count}\tValue of m: {m}, \tValue of c: {c}")

print(f"\nThe local minima occurs at: {m}, {c}")


Iteration: 1	Value of m: 0.8216742849995335, 	Value of c: -0.36773369499839675
Iteration: 2	Value of m: 0.7590265796907971, 	Value of c: -0.11982225937295118
Iteration: 3	Value of m: 0.7373610755254929, 	Value of c: -0.022508643098931004
Iteration: 4	Value of m: 0.7300225997319987, 	Value of c: 0.015733527105392834
Iteration: 5	Value of m: 0.7276072221606248, 	Value of c: 0.030779275066847574
Iteration: 6	Value of m: 0.726844957112927, 	Value of c: 0.036705755757572435
Iteration: 7	Value of m: 0.7266200788602646, 	Value of c: 0.03904297934951543
Iteration: 8	Value of m: 0.7265615750037493, 	Value of c: 0.0399658296919804
Iteration: 9	Value of m: 0.72655054523204, 	Value of c: 0.040330664326713846
Iteration: 10	Value of m: 0.7265510062489116, 	Value of c: 0.04047507506676204

The local minima occurs at: 0.7265510062489116, 0.04047507506676204


In [10]:

y_pred_train = []
for i in x_train:
    y_p_tr = (m * i) - c
    y_pred_train.append(y_p_tr)
y_pred_train = y_pred_train

In [12]:
y_pred_test = []
for i in x_test:
    y_p_te = (m * i) - c
    y_pred_test.append(y_p_te)
y_pred_test =y_pred_test

In [13]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
# train
mse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error', mse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error', mae)

Root mean square error 0.7123353408387804
Mean square error 0.5074216378079014
Mean absolute error 0.5486292016154564


In [14]:
#test
mse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error', mse) 
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error', mae)

Root mean square error 0.7259885373108695
Mean square error 0.5270593563067758
Mean absolute error 0.5527399600306773
